In [1]:
# SQL Executor Agent Example
"""
This notebook demonstrates creating an AI agent that uses the SQLExecutor
to execute SQL queries and provide insights on the results.
"""

from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient
from sql_executor import SQLExecutor
import json
from typing import Dict, Any

In [3]:
# Initialize the SQL Executor
# Update these paths to match your environment
data_path = "../data/bird/dev_databases"
dataset_name = "bird"

sql_executor = SQLExecutor(
    data_path=data_path,
    dataset_name=dataset_name
)

In [4]:
# Define SQL-related functions that the agent can use

async def execute_query(sql: str, db_id: str) -> str:
    """Execute a SQL query against a database."""
    result = sql_executor.safe_execute(sql, db_id)
    
    if result['success']:
        # Format successful result
        response = {
            "status": "success",
            "sql": result['sql'],
            "column_names": result['column_names'],
            "data": result['data'],
            "row_count": result['row_count'],
            "showing_rows": len(result['data'])
        }
    else:
        # Format error result
        response = {
            "status": "error",
            "sql": result['sql'],
            "error": result['sqlite_error'],
            "error_type": result['exception_class']
        }
        if result.get('timeout'):
            response['timeout'] = True
    
    return json.dumps(response, indent=2)

async def validate_query_result(sql: str, db_id: str) -> str:
    """Execute a query and validate its results."""
    result = sql_executor.safe_execute(sql, db_id)
    
    response = {
        "sql": result['sql'],
        "execution_success": result['success']
    }
    
    if result['success']:
        is_valid, reason = sql_executor.is_valid_result(result)
        response.update({
            "validation_result": "valid" if is_valid else "invalid",
            "validation_reason": reason,
            "row_count": result['row_count'],
            "has_null_values": "No" if is_valid and "NULL" not in reason else "Yes"
        })
    else:
        response.update({
            "validation_result": "execution_failed",
            "error": result['sqlite_error']
        })
    
    return json.dumps(response, indent=2)

async def test_queries(queries: list, db_id: str) -> str:
    """Test multiple SQL queries and summarize results."""
    results = []
    
    for sql in queries:
        result = sql_executor.safe_execute(sql, db_id)
        summary = {
            "sql": sql[:100] + "..." if len(sql) > 100 else sql,
            "success": result['success'],
            "row_count": result.get('row_count', 0) if result['success'] else None,
            "error": result.get('sqlite_error', '') if not result['success'] else None
        }
        results.append(summary)
    
    summary_stats = {
        "total_queries": len(queries),
        "successful": sum(1 for r in results if r['success']),
        "failed": sum(1 for r in results if not r['success']),
        "results": results
    }
    
    return json.dumps(summary_stats, indent=2)

async def explain_query_plan(sql: str, db_id: str) -> str:
    """Get the query execution plan for a SQL query."""
    # First try to get the query plan
    explain_sql = f"EXPLAIN QUERY PLAN {sql}"
    result = sql_executor.safe_execute(explain_sql, db_id)
    
    if result['success']:
        plan_info = {
            "sql": sql,
            "query_plan": result['data'],
            "analysis": "The query plan shows how SQLite will execute this query"
        }
    else:
        # If EXPLAIN fails, just execute the query normally
        normal_result = sql_executor.safe_execute(sql, db_id)
        if normal_result['success']:
            plan_info = {
                "sql": sql,
                "note": "Could not generate query plan, but query executes successfully",
                "row_count": normal_result['row_count']
            }
        else:
            plan_info = {
                "sql": sql,
                "error": normal_result['sqlite_error']
            }
    
    return json.dumps(plan_info, indent=2)

In [5]:
# Initialize the OpenAI model client
model_client = OpenAIChatCompletionClient(
    model="gpt-4o",
)

# Define the SQL execution agent with SQLExecutor tools
sql_agent = AssistantAgent(
    name="sql_agent",
    model_client=model_client,
    tools=[
        execute_query,
        validate_query_result,
        test_queries,
        explain_query_plan
    ],
    system_message="""You are a SQL expert assistant that helps users execute and analyze SQL queries.
    You can:
    - Execute SQL queries against databases and show results
    - Validate query results for potential issues
    - Test multiple queries and provide summaries
    - Explain query execution plans
    - Help debug SQL errors
    - Suggest query optimizations
    
    When executing queries, always show the results clearly and provide insights about the data.
    If a query fails, help users understand why and suggest fixes.""",
    reflect_on_tool_use=True,
    model_client_stream=True,
)

In [6]:
# Example: Execute a simple query
await Console(sql_agent.run_stream(
    task="Execute the query 'SELECT * FROM schools LIMIT 3' on the california_schools database"
))

---------- TextMessage (user) ----------
Execute the query 'SELECT * FROM schools LIMIT 3' on the california_schools database
---------- ToolCallRequestEvent (sql_agent) ----------
[FunctionCall(id='call_k95PPMQJdBh7zlmGk5Gkv1NI', arguments='{"sql":"SELECT * FROM schools LIMIT 3","db_id":"california_schools"}', name='execute_query')]
---------- ToolCallExecutionEvent (sql_agent) ----------
[FunctionExecutionResult(content='{\n  "status": "success",\n  "sql": "SELECT * FROM schools LIMIT 3",\n  "column_names": [\n    "CDSCode",\n    "NCESDist",\n    "NCESSchool",\n    "StatusType",\n    "County",\n    "District",\n    "School",\n    "Street",\n    "StreetAbr",\n    "City",\n    "Zip",\n    "State",\n    "MailStreet",\n    "MailStrAbr",\n    "MailCity",\n    "MailZip",\n    "MailState",\n    "Phone",\n    "Ext",\n    "Website",\n    "OpenDate",\n    "ClosedDate",\n    "Charter",\n    "CharterNum",\n    "FundingType",\n    "DOC",\n    "DOCType",\n    "SOC",\n    "SOCType",\n    "EdOpsCode

TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content="Execute the query 'SELECT * FROM schools LIMIT 3' on the california_schools database", type='TextMessage'), ToolCallRequestEvent(source='sql_agent', models_usage=RequestUsage(prompt_tokens=0, completion_tokens=0), metadata={}, content=[FunctionCall(id='call_k95PPMQJdBh7zlmGk5Gkv1NI', arguments='{"sql":"SELECT * FROM schools LIMIT 3","db_id":"california_schools"}', name='execute_query')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(source='sql_agent', models_usage=None, metadata={}, content=[FunctionExecutionResult(content='{\n  "status": "success",\n  "sql": "SELECT * FROM schools LIMIT 3",\n  "column_names": [\n    "CDSCode",\n    "NCESDist",\n    "NCESSchool",\n    "StatusType",\n    "County",\n    "District",\n    "School",\n    "Street",\n    "StreetAbr",\n    "City",\n    "Zip",\n    "State",\n    "MailStreet",\n    "MailStrAbr",\n    "MailCity",\n    "MailZip",\n    "MailState",\n   

In [ ]:
# Example: Validate a query with potential issues
await Console(sql_agent.run_stream(
    task="Validate the query 'SELECT School, AvgScrMath FROM satscores WHERE AvgScrMath IS NULL' on the california_schools database"
))

In [7]:
# Example: Test multiple queries
test_queries_list = [
    "SELECT COUNT(*) FROM schools",
    "SELECT AVG(AvgScrMath) FROM satscores WHERE AvgScrMath IS NOT NULL",
    "SELECT School, AvgScrMath FROM satscores ORDER BY AvgScrMath DESC LIMIT 5",
    "SELECT * FROM invalid_table"  # This should fail
]

await Console(sql_agent.run_stream(
    task=f"Test these queries on the california_schools database: {test_queries_list}"
))

---------- TextMessage (user) ----------
Test these queries on the california_schools database: ['SELECT COUNT(*) FROM schools', 'SELECT AVG(AvgScrMath) FROM satscores WHERE AvgScrMath IS NOT NULL', 'SELECT School, AvgScrMath FROM satscores ORDER BY AvgScrMath DESC LIMIT 5', 'SELECT * FROM invalid_table']
---------- ToolCallRequestEvent (sql_agent) ----------
[FunctionCall(id='call_jqhtnE0W0CkyXX5vUmjYM1Kr', arguments='{"queries":["SELECT COUNT(*) FROM schools","SELECT AVG(AvgScrMath) FROM satscores WHERE AvgScrMath IS NOT NULL","SELECT School, AvgScrMath FROM satscores ORDER BY AvgScrMath DESC LIMIT 5","SELECT * FROM invalid_table"],"db_id":"california_schools"}', name='test_queries')]
---------- ToolCallExecutionEvent (sql_agent) ----------
[FunctionExecutionResult(content='{\n  "total_queries": 4,\n  "successful": 2,\n  "failed": 2,\n  "results": [\n    {\n      "sql": "SELECT COUNT(*) FROM schools",\n      "success": true,\n      "row_count": 1,\n      "error": null\n    },\n    {\

TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content="Test these queries on the california_schools database: ['SELECT COUNT(*) FROM schools', 'SELECT AVG(AvgScrMath) FROM satscores WHERE AvgScrMath IS NOT NULL', 'SELECT School, AvgScrMath FROM satscores ORDER BY AvgScrMath DESC LIMIT 5', 'SELECT * FROM invalid_table']", type='TextMessage'), ToolCallRequestEvent(source='sql_agent', models_usage=RequestUsage(prompt_tokens=0, completion_tokens=0), metadata={}, content=[FunctionCall(id='call_jqhtnE0W0CkyXX5vUmjYM1Kr', arguments='{"queries":["SELECT COUNT(*) FROM schools","SELECT AVG(AvgScrMath) FROM satscores WHERE AvgScrMath IS NOT NULL","SELECT School, AvgScrMath FROM satscores ORDER BY AvgScrMath DESC LIMIT 5","SELECT * FROM invalid_table"],"db_id":"california_schools"}', name='test_queries')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(source='sql_agent', models_usage=None, metadata={}, content=[FunctionExecutionResult(content='{\n  "total_q

In [8]:
# Example: Explain query execution plan
await Console(sql_agent.run_stream(
    task="Explain the execution plan for 'SELECT s.School, sat.AvgScrMath FROM schools s JOIN satscores sat ON s.CDSCode = sat.cds' on california_schools database"
))

---------- TextMessage (user) ----------
Explain the execution plan for 'SELECT s.School, sat.AvgScrMath FROM schools s JOIN satscores sat ON s.CDSCode = sat.cds' on california_schools database
---------- ToolCallRequestEvent (sql_agent) ----------
[FunctionCall(id='call_YI8XpVrNPUq2hk8U8qV1WCtG', arguments='{"sql":"SELECT s.School, sat.AvgScrMath FROM schools s JOIN satscores sat ON s.CDSCode = sat.cds","db_id":"california_schools"}', name='explain_query_plan')]
---------- ToolCallExecutionEvent (sql_agent) ----------
[FunctionExecutionResult(content='{\n  "sql": "SELECT s.School, sat.AvgScrMath FROM schools s JOIN satscores sat ON s.CDSCode = sat.cds",\n  "query_plan": [\n    [\n      4,\n      0,\n      0,\n      "SCAN s"\n    ],\n    [\n      6,\n      0,\n      0,\n      "SEARCH sat USING INDEX sqlite_autoindex_satscores_1 (cds=?)"\n    ]\n  ],\n  "analysis": "The query plan shows how SQLite will execute this query"\n}', name='explain_query_plan', call_id='call_YI8XpVrNPUq2hk8U8qV

TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content="Explain the execution plan for 'SELECT s.School, sat.AvgScrMath FROM schools s JOIN satscores sat ON s.CDSCode = sat.cds' on california_schools database", type='TextMessage'), ToolCallRequestEvent(source='sql_agent', models_usage=RequestUsage(prompt_tokens=0, completion_tokens=0), metadata={}, content=[FunctionCall(id='call_YI8XpVrNPUq2hk8U8qV1WCtG', arguments='{"sql":"SELECT s.School, sat.AvgScrMath FROM schools s JOIN satscores sat ON s.CDSCode = sat.cds","db_id":"california_schools"}', name='explain_query_plan')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(source='sql_agent', models_usage=None, metadata={}, content=[FunctionExecutionResult(content='{\n  "sql": "SELECT s.School, sat.AvgScrMath FROM schools s JOIN satscores sat ON s.CDSCode = sat.cds",\n  "query_plan": [\n    [\n      4,\n      0,\n      0,\n      "SCAN s"\n    ],\n    [\n      6,\n      0,\n      0,\n      "SEARCH sat U

In [9]:
# Example: Debug a syntax error
await Console(sql_agent.run_stream(
    task="Help me fix this query: 'SELECT School FROM schools WHERE AvgScrMath > 500' on california_schools database. It's giving me an error."
))

---------- TextMessage (user) ----------
Help me fix this query: 'SELECT School FROM schools WHERE AvgScrMath > 500' on california_schools database. It's giving me an error.
---------- ModelClientStreamingChunkEvent (sql_agent) ----------
The error in your query likely stems from the fact that the `schools` table does not have a column named `AvgScrMath`. The `AvgScrMath` column is likely found in the `satscores` table instead.

To correct this, you need to join the `schools` and `satscores` tables on a common key (such as `CDSCode` for `schools` and the equivalent in `satscores`, possibly `cds`). Here's how you can modify the query:

```sql
SELECT s.School 
FROM schools s 
JOIN satscores sat ON s.CDSCode = sat.cds
WHERE sat.AvgScrMath > 500
```

This query will retrieve the school names from the `schools` table where the average math score from the `satscores` table is greater than 500. It assumes that `CDSCode` in the `schools` table corresponds to `cds` in the `satscores` table. Mak

TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content="Help me fix this query: 'SELECT School FROM schools WHERE AvgScrMath > 500' on california_schools database. It's giving me an error.", type='TextMessage'), TextMessage(source='sql_agent', models_usage=RequestUsage(prompt_tokens=0, completion_tokens=0), metadata={}, content="The error in your query likely stems from the fact that the `schools` table does not have a column named `AvgScrMath`. The `AvgScrMath` column is likely found in the `satscores` table instead.\n\nTo correct this, you need to join the `schools` and `satscores` tables on a common key (such as `CDSCode` for `schools` and the equivalent in `satscores`, possibly `cds`). Here's how you can modify the query:\n\n```sql\nSELECT s.School \nFROM schools s \nJOIN satscores sat ON s.CDSCode = sat.cds\nWHERE sat.AvgScrMath > 500\n```\n\nThis query will retrieve the school names from the `schools` table where the average math score from the `s

In [10]:
# Advanced Example: Create a more sophisticated SQL assistant with analysis capabilities

class SQLAnalysisAgent(AssistantAgent):
    """An advanced SQL analysis agent with additional analytical capabilities."""
    
    def __init__(self, sql_executor: SQLExecutor, **kwargs):
        self.sql_executor = sql_executor
        
        # Define advanced analysis functions
        async def analyze_table_statistics(table_name: str, db_id: str) -> str:
            """Analyze statistics for a specific table."""
            queries = {
                "row_count": f"SELECT COUNT(*) as count FROM {table_name}",
                "null_analysis": f"SELECT * FROM {table_name} WHERE 1=0",  # Get column names
            }
            
            stats = {"table": table_name}
            
            # Get row count
            count_result = self.sql_executor.safe_execute(queries["row_count"], db_id)
            if count_result['success']:
                stats["total_rows"] = count_result['data'][0][0]
            
            # Get column information
            col_result = self.sql_executor.safe_execute(queries["null_analysis"], db_id)
            if col_result['success']:
                columns = col_result['column_names']
                stats["column_count"] = len(columns)
                stats["columns"] = columns
                
                # Analyze NULL values for each column
                null_counts = {}
                for col in columns[:5]:  # Limit to first 5 columns for performance
                    null_query = f"SELECT COUNT(*) FROM {table_name} WHERE {col} IS NULL"
                    null_result = self.sql_executor.safe_execute(null_query, db_id)
                    if null_result['success']:
                        null_counts[col] = null_result['data'][0][0]
                
                stats["null_counts"] = null_counts
            
            return json.dumps(stats, indent=2)
        
        async def compare_query_performance(queries: list, db_id: str) -> str:
            """Compare performance of multiple queries."""
            import time
            
            results = []
            for sql in queries:
                start_time = time.time()
                result = self.sql_executor.safe_execute(sql, db_id)
                execution_time = time.time() - start_time
                
                perf_info = {
                    "query": sql[:100] + "..." if len(sql) > 100 else sql,
                    "success": result['success'],
                    "execution_time_seconds": round(execution_time, 3),
                    "row_count": result.get('row_count', 0) if result['success'] else None,
                    "error": result.get('sqlite_error') if not result['success'] else None
                }
                results.append(perf_info)
            
            # Sort by execution time
            results.sort(key=lambda x: x['execution_time_seconds'])
            
            return json.dumps({
                "performance_comparison": results,
                "fastest_query": results[0]['query'] if results else None,
                "slowest_query": results[-1]['query'] if results else None
            }, indent=2)
        
        async def suggest_indexes(table_name: str, db_id: str) -> str:
            """Suggest potential indexes for a table based on its structure."""
            # Get table structure
            pragma_query = f"PRAGMA table_info({table_name})"
            result = self.sql_executor.safe_execute(pragma_query, db_id)
            
            suggestions = []
            if result['success']:
                columns = result['data']
                for col in columns:
                    col_name = col[1]
                    col_type = col[2]
                    
                    # Suggest indexes for common patterns
                    if col_name.lower().endswith('_id'):
                        suggestions.append({
                            "column": col_name,
                            "reason": "Foreign key column - frequently used in JOINs",
                            "index_sql": f"CREATE INDEX idx_{table_name}_{col_name} ON {table_name}({col_name});"
                        })
                    elif col_name.lower() in ['created_at', 'updated_at', 'date']:
                        suggestions.append({
                            "column": col_name,
                            "reason": "Date column - often used in WHERE clauses for filtering",
                            "index_sql": f"CREATE INDEX idx_{table_name}_{col_name} ON {table_name}({col_name});"
                        })
                    elif col_type in ['INTEGER', 'REAL'] and not col_name.lower().endswith('id'):
                        suggestions.append({
                            "column": col_name,
                            "reason": "Numeric column - potentially used in range queries",
                            "index_sql": f"CREATE INDEX idx_{table_name}_{col_name} ON {table_name}({col_name});"
                        })
            
            return json.dumps({
                "table": table_name,
                "index_suggestions": suggestions[:3]  # Limit to top 3 suggestions
            }, indent=2)
        
        # Initialize with enhanced tools
        super().__init__(
            tools=[
                execute_query,
                validate_query_result,
                analyze_table_statistics,
                compare_query_performance,
                suggest_indexes,
                explain_query_plan
            ],
            system_message="""You are an advanced SQL analysis expert that helps users:
            - Execute and debug SQL queries
            - Analyze table statistics and data quality
            - Compare query performance
            - Suggest database optimizations and indexes
            - Provide insights about query results
            
            Always provide clear explanations and actionable recommendations.""",
            **kwargs
        )

# Create the advanced SQL analysis agent
sql_analysis_agent = SQLAnalysisAgent(
    sql_executor=sql_executor,
    name="sql_analysis_agent",
    model_client=model_client,
    reflect_on_tool_use=True,
    model_client_stream=True,
)

In [11]:
# Test advanced agent: Analyze table statistics
await Console(sql_analysis_agent.run_stream(
    task="Analyze the statistics for the 'schools' table in california_schools database"
))

---------- TextMessage (user) ----------
Analyze the statistics for the 'schools' table in california_schools database
---------- ToolCallRequestEvent (sql_analysis_agent) ----------
[FunctionCall(id='call_NzUfofgJcHFAmV2lDwcfGYRR', arguments='{"table_name":"schools","db_id":"california_schools"}', name='analyze_table_statistics')]
---------- ToolCallExecutionEvent (sql_analysis_agent) ----------
[FunctionExecutionResult(content='{\n  "table": "schools",\n  "total_rows": 17686,\n  "column_count": 49,\n  "columns": [\n    "CDSCode",\n    "NCESDist",\n    "NCESSchool",\n    "StatusType",\n    "County",\n    "District",\n    "School",\n    "Street",\n    "StreetAbr",\n    "City",\n    "Zip",\n    "State",\n    "MailStreet",\n    "MailStrAbr",\n    "MailCity",\n    "MailZip",\n    "MailState",\n    "Phone",\n    "Ext",\n    "Website",\n    "OpenDate",\n    "ClosedDate",\n    "Charter",\n    "CharterNum",\n    "FundingType",\n    "DOC",\n    "DOCType",\n    "SOC",\n    "SOCType",\n    "EdOp

TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content="Analyze the statistics for the 'schools' table in california_schools database", type='TextMessage'), ToolCallRequestEvent(source='sql_analysis_agent', models_usage=RequestUsage(prompt_tokens=0, completion_tokens=0), metadata={}, content=[FunctionCall(id='call_NzUfofgJcHFAmV2lDwcfGYRR', arguments='{"table_name":"schools","db_id":"california_schools"}', name='analyze_table_statistics')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(source='sql_analysis_agent', models_usage=None, metadata={}, content=[FunctionExecutionResult(content='{\n  "table": "schools",\n  "total_rows": 17686,\n  "column_count": 49,\n  "columns": [\n    "CDSCode",\n    "NCESDist",\n    "NCESSchool",\n    "StatusType",\n    "County",\n    "District",\n    "School",\n    "Street",\n    "StreetAbr",\n    "City",\n    "Zip",\n    "State",\n    "MailStreet",\n    "MailStrAbr",\n    "MailCity",\n    "MailZip",\n    "MailState",

In [12]:
# Test advanced agent: Compare query performance
performance_queries = [
    "SELECT COUNT(*) FROM schools",
    "SELECT * FROM schools WHERE County = 'Los Angeles'",
    "SELECT s.School, sat.AvgScrMath FROM schools s JOIN satscores sat ON s.CDSCode = sat.cds"
]

await Console(sql_analysis_agent.run_stream(
    task=f"Compare the performance of these queries on california_schools database: {performance_queries}"
))

---------- TextMessage (user) ----------
Compare the performance of these queries on california_schools database: ['SELECT COUNT(*) FROM schools', "SELECT * FROM schools WHERE County = 'Los Angeles'", 'SELECT s.School, sat.AvgScrMath FROM schools s JOIN satscores sat ON s.CDSCode = sat.cds']
---------- ToolCallRequestEvent (sql_analysis_agent) ----------
[FunctionCall(id='call_npn0Clak1boVvomA91VyqMXe', arguments='{"queries":["SELECT COUNT(*) FROM schools","SELECT * FROM schools WHERE County = \'Los Angeles\'","SELECT s.School, sat.AvgScrMath FROM schools s JOIN satscores sat ON s.CDSCode = sat.cds"],"db_id":"california_schools"}', name='compare_query_performance')]
---------- ToolCallExecutionEvent (sql_analysis_agent) ----------
[FunctionExecutionResult(content='{\n  "performance_comparison": [\n    {\n      "query": "SELECT COUNT(*) FROM schools",\n      "success": true,\n      "execution_time_seconds": 0.001,\n      "row_count": 1,\n      "error": null\n    },\n    {\n      "query"

TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='Compare the performance of these queries on california_schools database: [\'SELECT COUNT(*) FROM schools\', "SELECT * FROM schools WHERE County = \'Los Angeles\'", \'SELECT s.School, sat.AvgScrMath FROM schools s JOIN satscores sat ON s.CDSCode = sat.cds\']', type='TextMessage'), ToolCallRequestEvent(source='sql_analysis_agent', models_usage=RequestUsage(prompt_tokens=0, completion_tokens=0), metadata={}, content=[FunctionCall(id='call_npn0Clak1boVvomA91VyqMXe', arguments='{"queries":["SELECT COUNT(*) FROM schools","SELECT * FROM schools WHERE County = \'Los Angeles\'","SELECT s.School, sat.AvgScrMath FROM schools s JOIN satscores sat ON s.CDSCode = sat.cds"],"db_id":"california_schools"}', name='compare_query_performance')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(source='sql_analysis_agent', models_usage=None, metadata={}, content=[FunctionExecutionResult(content='{\n  "performance_c

In [13]:
# Test advanced agent: Suggest indexes
await Console(sql_analysis_agent.run_stream(
    task="Suggest indexes for the 'satscores' table in california_schools database"
))

---------- TextMessage (user) ----------
Suggest indexes for the 'satscores' table in california_schools database
---------- ToolCallRequestEvent (sql_analysis_agent) ----------
[FunctionCall(id='call_DCnrEJHYEUZc3uZURDKhoTur', arguments='{"table_name":"satscores","db_id":"california_schools"}', name='suggest_indexes')]
---------- ToolCallExecutionEvent (sql_analysis_agent) ----------
[FunctionExecutionResult(content='{\n  "table": "satscores",\n  "index_suggestions": []\n}', name='suggest_indexes', call_id='call_DCnrEJHYEUZc3uZURDKhoTur', is_error=False)]
---------- ModelClientStreamingChunkEvent (sql_analysis_agent) ----------
Currently, there are no specific index suggestions automatically generated for the `satscores` table in the `california_schools` database. However, here are some general recommendations for considering index creation:

1. **Join Columns**: If `satscores` is regularly joined with `schools` using a `cds` (or similar) column, as referenced in the previous query, e

TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content="Suggest indexes for the 'satscores' table in california_schools database", type='TextMessage'), ToolCallRequestEvent(source='sql_analysis_agent', models_usage=RequestUsage(prompt_tokens=0, completion_tokens=0), metadata={}, content=[FunctionCall(id='call_DCnrEJHYEUZc3uZURDKhoTur', arguments='{"table_name":"satscores","db_id":"california_schools"}', name='suggest_indexes')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(source='sql_analysis_agent', models_usage=None, metadata={}, content=[FunctionExecutionResult(content='{\n  "table": "satscores",\n  "index_suggestions": []\n}', name='suggest_indexes', call_id='call_DCnrEJHYEUZc3uZURDKhoTur', is_error=False)], type='ToolCallExecutionEvent'), TextMessage(source='sql_analysis_agent', models_usage=RequestUsage(prompt_tokens=0, completion_tokens=0), metadata={}, content="Currently, there are no specific index suggestions automatically generated f

In [14]:
# Close the model client
await model_client.close()